In [18]:
import pandas as pd
import numpy as np

In [19]:
# 캐시플로 데이터
cash_flow = pd.read_csv("../raw_data/CASHFLOW_REAL.csv", thousands=",", encoding="CP949")

# 이 아래는 괜찮고
self_jabon = pd.read_csv("../raw_data/자기자본구성비율_Raw.csv", thousands=",", encoding="CP949")
jego = pd.read_csv("../raw_data/재고자산대유동자산비율_Raw.csv", thousands=",", encoding="CP949")
udong = pd.read_csv("../raw_data/유동자산구성비율_Raw.csv", thousands=",", encoding="CP949")
chaip = pd.read_csv("../raw_data/차입금평균이자율_Raw.csv", thousands=",", encoding="CP949")
finance_fee = pd.read_csv("../raw_data/금융비용부담률_Raw.csv", thousands=",", encoding="CP949")
income = pd.read_csv("../raw_data/매출원가대매출액비율_Raw.csv", thousands=",", encoding="CP949")

In [20]:
cash_flow.head(1)

,회사명,거래소코드,회계년도,[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS),[제조]매출액(수익)(*)(IFRS),[제조]부채(*)(IFRS),[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결),[제조]매출액(수익)(*)(IFRS연결),[제조]부채(*)(IFRS연결)
0,(주)CMG제약,58820,2010/12,-5983864.0,11350419.0,8370511.0,-5983864.0,11350419.0,8370511.0


In [21]:
# cashflow shift 처리

cash_flow["[제조]영업활동현금흐름(개별)"] = cash_flow["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS)"].shift().where(cash_flow.회사명.eq(cash_flow.회사명.shift()))
cash_flow["[제조]영업활동현금흐름(연결)"] = cash_flow["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)"].shift().where(cash_flow.회사명.eq(cash_flow.회사명.shift()))

In [22]:
cash_flow.head(1)

,회사명,거래소코드,회계년도,[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS),[제조]매출액(수익)(*)(IFRS),[제조]부채(*)(IFRS),[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결),[제조]매출액(수익)(*)(IFRS연결),[제조]부채(*)(IFRS연결),[제조]영업활동현금흐름(개별),[제조]영업활동현금흐름(연결)
0,(주)CMG제약,58820,2010/12,-5983864.0,11350419.0,8370511.0,-5983864.0,11350419.0,8370511.0,NaN,NaN


In [23]:
# # cashflow 대 부채 비율

# for i in range(0, len(cash_flow)):
#     if ((cash_flow.iloc[i]["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS)"] + cash_flow.iloc[i]["[제조]영업활동현금흐름(개별)"])/2) < 0 and cash_flow.iloc[i]["[제조]부채(*)(IFRS)"] < 0 :
#         cash_flow.loc[i, "cashflow_부채비율(개별)"] = (((cash_flow.iloc[i]["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS)"] + cash_flow.iloc[i]["[제조]영업활동현금흐름(개별)"])/2)/cash_flow.iloc[i]["[제조]부채(*)(IFRS)"]) * -1
#     else:
#         cash_flow.loc[i, "cashflow_부채비율(개별)"] = ((cash_flow.iloc[i]["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS)"] + cash_flow.iloc[i]["[제조]영업활동현금흐름(개별)"])/2)/cash_flow.iloc[i]["[제조]부채(*)(IFRS)"]

#     if ((cash_flow.iloc[i]["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)"] - cash_flow.iloc[i]["[제조]영업활동현금흐름(연결)"])/2) < 0 and cash_flow.iloc[i]["[제조]당기순이익(손실)(IFRS연결)"] < 0 :
#         cash_flow.loc[i, "cashflow_부채비율(연결)"] = (((cash_flow.iloc[i]["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)"] + cash_flow.iloc[i]["[제조]영업활동현금흐름(연결)"])/2)/cash_flow.iloc[i]["[제조]부채(*)(IFRS)"]) * -1
#     else:
#         cash_flow.loc[i, "cashflow_부채비율(연결)"] = ((cash_flow.iloc[i]["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)"] + cash_flow.iloc[i]["[제조]영업활동현금흐름(연결)"])/2)/cash_flow.iloc[i]["[제조]부채(*)(IFRS)"]

# # cashflow 대 매출 비율

# for i in range(0, len(cash_flow)):
#     if ((cash_flow.iloc[i]["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS)"] + cash_flow.iloc[i]["[제조]영업활동현금흐름(개별)"])/2) < 0 and cash_flow.iloc[i]["[제조]당기순이익(손실)(IFRS)"] < 0 :
#         cash_flow.loc[i, "cashflow_매출비율(개별)"] = (((cash_flow.iloc[i]["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS)"] + cash_flow.iloc[i]["[제조]영업활동현금흐름(개별)"])/2)/cash_flow.iloc[i]["[제조]매출액(수익)(*)(IFRS)"]) * -1
#     else:
#         cash_flow.loc[i, "cashflow_매출비율(개별)"] = ((cash_flow.iloc[i]["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS)"] + cash_flow.iloc[i]["[제조]영업활동현금흐름(개별)"])/2)/cash_flow.iloc[i]["[제조]매출액(수익)(*)(IFRS)"]
    
#     if cash_flow.iloc[i]["[제조]자본(*)(IFRS연결)"] < 0 and cash_flow.iloc[i]["[제조]당기순이익(손실)(IFRS연결)"] < 0 :
#         cash_flow.loc[i, "cashflow_매출비율(연결)"] = (((cash_flow.iloc[i]["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)"] + cash_flow.iloc[i]["[제조]영업활동현금흐름(연결)"])/2)/cash_flow.iloc[i]["[제조]매출액(수익)(*)(IFRS연결)"]) * -1
#     else:
#         cash_flow.loc[i, "cashflow_매출비율(연결)"] = ((cash_flow.iloc[i]["[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)"] + cash_flow.iloc[i]["[제조]영업활동현금흐름(연결)"])/2)/cash_flow.iloc[i]["[제조]매출액(수익)(*)(IFRS연결)"]

In [27]:
self_jabon.head(1)

,회사명,거래소코드,회계년도,자본(IFRS연결),자산(IFRS연결),자본(IFRS개별),자산(IFRS개별),자기자본구성비율(연결),자기자본구성비율(개별)
0,(주)CMG제약,58820,Dec-11,18034249.0,27305198.0,18290915.0,27548539.0,66.046945,66.395227


In [25]:
# 자기자본구성비율

for i in range(0, len(self_jabon)):
    if self_jabon.iloc[i]["자본(IFRS연결)"] < 0 and self_jabon.iloc[i]["자산(IFRS연결)"] < 0 :
        self_jabon.loc[i, "자기자본구성비율(연결)"] = self_jabon.iloc[i]["자본(IFRS연결)"]/self_jabon.iloc[i]["자산(IFRS연결)"] * -100
    else:
        self_jabon.loc[i, "자기자본구성비율(연결)"] = self_jabon.iloc[i]["자본(IFRS연결)"]/self_jabon.iloc[i]["자산(IFRS연결)"] * 100

    if self_jabon.iloc[i]["자본(IFRS개별)"] < 0 and self_jabon.iloc[i]["자본(IFRS개별)"] < 0 :
        self_jabon.loc[i, "자기자본구성비율(개별)"] = self_jabon.iloc[i]["자본(IFRS개별)"]/self_jabon.iloc[i]["자산(IFRS개별)"] * -100
    else:
        self_jabon.loc[i, "자기자본구성비율(개별)"] = self_jabon.iloc[i]["자본(IFRS개별)"]/self_jabon.iloc[i]["자산(IFRS개별)"] * 100

In [26]:
jego.head(1)

,회사명,거래소코드,회계년도,재고자산(IFRS연결),유동자산(IFRS연결),재고자산(IFRS개별),유동자산(IFRS개별)
0,(주)CMG제약,58820,Dec-11,2743103.0,15341167.0,2743103.0,15264885.0


In [ ]:
# 재고자산 대 유동자산 비율

for i in range(0, len(self_jabon)):
    if self_jabon.iloc[i]["재고자산(IFRS연결)"] < 0 and self_jabon.iloc[i]["유동자산(IFRS연결)"] < 0 :
        self_jabon.loc[i, "재고자산유동비율(연결)"] = self_jabon.iloc[i]["재고자산(IFRS연결)"]/self_jabon.iloc[i]["유동자산(IFRS연결)"] * -100
    else:
        self_jabon.loc[i, "재고자산유동비율(연결)"] = self_jabon.iloc[i]["재고자산(IFRS연결)"]/self_jabon.iloc[i]["유동자산(IFRS연결)"] * 100

    if self_jabon.iloc[i]["재고자산(IFRS개별)"] < 0 and self_jabon.iloc[i]["유동자산(IFRS개별)"] < 0 :
        self_jabon.loc[i, "재고자산유동비율(개별)"] = self_jabon.iloc[i]["재고자산(IFRS개별)"]/self_jabon.iloc[i]["유동자산(IFRS개별)"] * -100
    else:
        self_jabon.loc[i, "재고자산유동비율(개별)"] = self_jabon.iloc[i]["재고자산(IFRS개별)"]/self_jabon.iloc[i]["유동자산(IFRS개별)"] * 100

In [ ]:
udong.head(1)

,회사명,거래소코드,회계년도,유동자산(IFRS연결),자산(IFRS연결),유동자산(IFRS개별),자산(IFRS개별)
0,(주)CMG제약,58820,Dec-11,15341167.0,27305198.0,15264885.0,27548539.0


In [ ]:
# 유동자산 구성비율

for i in range(0, len(udong)):
    if udong.iloc[i]["유동자산(IFRS연결)"] < 0 and udong.iloc[i]["자산(IFRS연결)"] < 0 :
        udong.loc[i, "유동자산구성비율(연결)"] = udong.iloc[i]["유동자산(IFRS연결)"]/udong.iloc[i]["자산(IFRS연결)"] * -100
    else:
        udong.loc[i, "유동자산구성비율(연결)"] = udong.iloc[i]["유동자산(IFRS연결)"]/udong.iloc[i]["자산(IFRS연결)"] * 100

    if udong.iloc[i]["유동자산(IFRS개별)"] < 0 and udong.iloc[i]["자산(IFRS개별)"] < 0 :
        udong.loc[i, "유동자산구성비율(개별)"] = udong.iloc[i]["유동자산(IFRS개별)"]/udong.iloc[i]["자산(IFRS개별)"] * -100
    else:
        udong.loc[i, "유동자산구성비율(개별)"] = udong.iloc[i]["유동자산(IFRS개별)"]/udong.iloc[i]["자산(IFRS개별)"] * 100

In [ ]:
chaip.head(1)

,회사명,거래소코드,회계년도,장기차입금(IFRS연결),단기차입금(IFRS연결),유동성장기부채(IFRS연결),이자비용(IFRS연결),장기차입금(IFRS개별),단기차입금(IFRS개별),유동성장기부채(IFRS개별),이자비용(IFRS개별)
0,(주)CMG제약,58820,Dec-11,0.0,0.0,2000000.0,0.0,0.0,2134049.0,2000000.0,0.0


In [ ]:
# 차입금 평균 이자율

for i in range(0, len(chaip)):
    if chaip.iloc[i]["이자비용(IFRS연결)"] < 0 and ((chaip.iloc[i]["장기차입금(IRFS연결)"]+chaip.iloc[i]["단기차입금(IFRS연결)"]+chaip.iloc[i]["유동성장기부채(IFRS연결)"])/3) < 0 :
        chaip.loc[i, "차입금평균이자율(연결)"] = chaip.iloc[i]["이자비용(IFRS연결)"]/((chaip.iloc[i]["장기차입금(IRFS연결)"]+chaip.iloc[i]["단기차입금(IFRS연결)"]+chaip.iloc[i]["유동성장기부채(IFRS연결)"])/3) * -100
    else:
        chaip.loc[i, "차입금평균이자율(연결)"] = ((chaip.iloc[i]["장기차입금(IRFS연결)"]+chaip.iloc[i]["단기차입금(IFRS연결)"]+chaip.iloc[i]["유동성장기부채(IFRS연결)"])/3) * 100

    if chaip.iloc[i]["이자비용(IFRS개별)"] < 0 and ((chaip.iloc[i]["장기차입금(IRFS개별)"]+chaip.iloc[i]["단기차입금(IFRS개별)"]+chaip.iloc[i]["유동성장기부채(IFRS개별)"])/3) < 0 :
        chaip.loc[i, "차입금평균이자율(개별)"] = chaip.iloc[i]["유동자산(IFRS개별)"]/((chaip.iloc[i]["장기차입금(IRFS개별)"]+chaip.iloc[i]["단기차입금(IFRS개별)"]+chaip.iloc[i]["유동성장기부채(IFRS개별)"])/3) * -100
    else:
        chaip.loc[i, "차입금평균이자율(개별)"] = chaip.iloc[i]["유동자산(IFRS개별)"]/((chaip.iloc[i]["장기차입금(IRFS개별)"]+chaip.iloc[i]["단기차입금(IFRS개별)"]+chaip.iloc[i]["유동성장기부채(IFRS개별)"])/3) * 100

In [ ]:
finance_fee.head(1)

,회사명,거래소코드,회계년도,[제조] 금융비용(IFRS연결),[제조]매출액(수익)(*)(IFRS연결),[제조] 금융비용(IFRS),[제조]매출액(수익)(*)(IFRS)
0,(주)CMG제약,58820,2011/12,0.0,17751704.0,0.0,17751704.0


In [ ]:
# 금융비용 부담률

for i in range(0, len(finance_fee)):
    if finance_fee.iloc[i]["[제조] 금융비용(IFRS연결)"] < 0 and finance_fee.iloc[i]["[제조]매출액(수익)(*)(IFRS연결)"] < 0 :
        finance_fee.loc[i, "금융비용부담률(연결)"] = finance_fee.iloc[i]["[제조] 금융비용(IFRS연결)"]/finance_fee.iloc[i]["[제조]매출액(수익)(*)(IFRS연결)"] * -100
    else:
        finance_fee.loc[i, "금융비용부담률(연결)"] = finance_fee.iloc[i]["[제조] 금융비용(IFRS연결)"]/finance_fee.iloc[i]["[제조]매출액(수익)(*)(IFRS연결)"] * 100

    if finance_fee.iloc[i]["[제조] 금융비용(IFRS)"] < 0 and finance_fee.iloc[i]["[제조]매출액(수익)(*)(IFRS)"] < 0 :
        finance_fee.loc[i, "금융비용부담률(개별)"] = finance_fee.iloc[i]["[제조] 금융비용(IFRS)"]/finance_fee.iloc[i]["[제조]매출액(수익)(*)(IFRS)"] * -100
    else:
        finance_fee.loc[i, "금융비용부담률(개별)"] = finance_fee.iloc[i]["[제조] 금융비용(IFRS)"]/finance_fee.iloc[i]["[제조]매출액(수익)(*)(IFRS)"] * 100

In [ ]:
income.head(1)

,회사명,거래소코드,회계년도,[제조]매출원가(*)(IFRS연결),[제조]매출액(수익)(*)(IFRS연결),[제조]매출원가(*)(IFRS),[제조]매출액(수익)(*)(IFRS),매출원가_매출액비율(연결),매출원가_매출액비율(개별)
0,(주)CMG제약,58820,2011/12,10325799.0,17751704.0,10325799.0,17751704.0,58.167931,58.167931


In [ ]:
# 매출원가 대 매출액 비율

for i in range(0, len(income)):
    if income.iloc[i]["[제조]매출원가(*)(IFRS연결)"] < 0 and income.iloc[i]["[제조]매출액(수익)(*)(IFRS연결)"] < 0 :
        income.loc[i, "매출원가_매출액비율(연결)"] = income.iloc[i]["[제조]매출원가(*)(IFRS연결)"]/income.iloc[i]["[제조]매출액(수익)(*)(IFRS연결)"] * -100
    else:
        income.loc[i, "매출원가_매출액비율(연결)"] = income.iloc[i]["[제조]매출원가(*)(IFRS연결)"]/income.iloc[i]["[제조]매출액(수익)(*)(IFRS연결)"] * 100

    if income.iloc[i]["[제조]매출원가(*)(IFRS)"] < 0 and income.iloc[i]["[제조]매출액(수익)(*)(IFRS)"] < 0 :
        income.loc[i, "매출원가_매출액비율(개별)"] = income.iloc[i]["[제조]매출원가(*)(IFRS)"]/income.iloc[i]["[제조]매출액(수익)(*)(IFRS)"] * -100
    else:
        income.loc[i, "매출원가_매출액비율(개별)"] = income.iloc[i]["[제조]매출원가(*)(IFRS)"]/income.iloc[i]["[제조]매출액(수익)(*)(IFRS)"] * 100

C:\Users\luck1\AppData\Local\Temp\ipykernel_12076\3723378413.py:12: RuntimeWarning: invalid value encountered in double_scalars
  income.loc[i, "매출원가_매출액비율(개별)"] = income.iloc[i]["[제조]매출원가(*)(IFRS)"]/income.iloc[i]["[제조]매출액(수익)(*)(IFRS)"] * 100
C:\Users\luck1\AppData\Local\Temp\ipykernel_12076\3723378413.py:7: RuntimeWarning: invalid value encountered in double_scalars
  income.loc[i, "매출원가_매출액비율(연결)"] = income.iloc[i]["[제조]매출원가(*)(IFRS연결)"]/income.iloc[i]["[제조]매출액(수익)(*)(IFRS연결)"] * 100
